In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import torchvision.models as models
import matplotlib.pyplot as plt
from torch import nn
!pip install torchinfo
from torchinfo import summary
from torch.nn import functional as F

## 하이퍼 파라미터 정의 

In [2]:
learning_rate = 0.001
momentum = 0.9
training_epochs = 20
batch_size = 100
torch.manual_seed(2018171013)

## 데이터 가져오기 

In [3]:
transforms_train = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    #transforms.RandomHorizontalFlip(p=0.5),
    #transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])
trainset = torchvision.datasets.ImageFolder(root="../input/chest-xray-pneumonia/chest_xray/train", transform=transforms_train)
testset = torchvision.datasets.ImageFolder(root="../input/chest-xray-pneumonia/chest_xray/test", transform=transforms_test)


## 데이터 로더

In [4]:
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

## 데이터 체크 

In [5]:
print(trainset.__getitem__(0)[0].size(), trainset.__len__())
print(testset.__getitem__(0)[0].size(), testset.__len__())

print(len(trainset),len(testset))

torch.Size([3, 224, 224]) 5216
torch.Size([3, 224, 224]) 624
5216 624


## 모델 정의 

In [6]:
#모델을 정의할 때 유념해야할 건 각 층의 결과물의 사이즈입니다. 
#이 층에서 어떤 size를 가진 데이터를 몇 개 도출하는지 정확하게 알고 있어야 정확한 모델을 정의할 수 있습니다. 
#모델이 복잡해지면 복잡해질수록 이는 더욱 중요해집니다. 때문에 모델을 정의할 때 옆에 각 모델이 도출하는 데이터의 크기를 써넣는 것을 추천드립니다. 
#해당 모델은 임의로 만든 모델입니다. 이 모델이 다른 모델보다 뛰어나다는 보장이 없습니다. 
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()           
        self.layer = nn.Sequential(                                             
            nn.Conv2d(in_channels=3,out_channels=16,kernel_size=5),             # [batch_size,3,224,224] -> [batch_size,16,220,220]
            nn.ReLU(),                                                          
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5),            #  [batch_size,32,216,216]
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),                               # [batch_size,32,108,108] 
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),          # [batch_size,64,104,104] 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)                                # [batch_size,64,52,52] 
        )
        self.fc_layer = nn.Sequential(                                          
            nn.Linear(64*52*52,100),             #Linear(64*52*52,100)                                   # [batch_size,64*52*52] -> [batch_size,100]
            nn.ReLU(),
            nn.Linear(100,1)                                                   # [batch_size,100] -> [batch_size,1]
        )       
        
    def forward(self,x):
        out = self.layer(x)                                                     # self.layer에 정의한 Sequential의 연산을 차례대로 다 실행합니다.
        out = out.view(-1,64*52*52)                                               # view 함수를 이용해 텐서의 형태를 [batch_size,나머지]로 바꿔줍니다. 
                                                                                # ex) 2x3 형태였던 텐서를 .view(1,-1) 해주면 1x6의 형태로 바뀝니다. .view(3,-1)이면 3x2로 바뀜.
                                                                                # 만약 전체 텐서의 크기가 batch_size로 나누어 떨어지지 않으면 오류가 납니다.
        out = self.fc_layer(out)
    
        return torch.sigmoid(out)


# https://pytorch.org/docs/stable/nn.html?highlight=conv2d#torch.nn.Conv2d
# https://pytorch.org/docs/stable/tensors.html?highlight=view#torch.Tensor.view

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
Model = CNN().to(device)
print(summary(Model, input_size=(1, 3, 224, 224), verbose=0))

  

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      --                        --
├─Sequential: 1-1                        [1, 64, 52, 52]           --
│    └─Conv2d: 2-1                       [1, 16, 220, 220]         1,216
│    └─ReLU: 2-2                         [1, 16, 220, 220]         --
│    └─Conv2d: 2-3                       [1, 32, 216, 216]         12,832
│    └─ReLU: 2-4                         [1, 32, 216, 216]         --
│    └─MaxPool2d: 2-5                    [1, 32, 108, 108]         --
│    └─Conv2d: 2-6                       [1, 64, 104, 104]         51,264
│    └─ReLU: 2-7                         [1, 64, 104, 104]         --
│    └─MaxPool2d: 2-8                    [1, 64, 52, 52]           --
├─Sequential: 1-2                        [1, 1]                    --
│    └─Linear: 2-9                       [1, 100]                  17,305,700
│    └─ReLU: 2-10                        [1, 100]                 

Loss ,optim 정의 

In [7]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(Model.parameters(), momentum=momentum, lr=learning_rate)

total_data = len(trainset) # 5216 
iteration_num = len(trainloader)

print("LEARNING STARTS! (model: My CNN model)")
print("total data is ", total_data)
print("there will be about ", iteration_num, "steps")

LEARNING STARTS! (model: My CNN model)
total data is  5216
there will be about  53 steps


학습 ,테스팅 

In [8]:
current_accuracy = 0
for epoch in range(training_epochs):
    
    avg_cost = 0
    step = 0
    for X, Y in trainloader:
        X = X.to(device)
        Y =Y.view(-1,1)
        Y =Y.to(torch.float32)
        Y =Y.to(device)
        optimizer.zero_grad()
        hypothesis = Model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost / iteration_num
        if(step % 100 == 0):
            print("STEP [", step, "/", iteration_num, "] LOSS: ", cost.item())
        step += 1
    print('[EPOCH: {:>4}] COST = {:>.9}'.format(epoch + 1, avg_cost))
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            outputs = Model(images.to(device))
            total += labels.size(0)
            #correct_prediction = torch.argmax(outputs.data, 1) == labels.to(device)
            correct_prediction = outputs[outputs.data>0.5]
            correct += correct_prediction.sum()

        accuracy = int(correct) / total
        print('EPOCH', epoch+1,  ' ACCURACY:', accuracy)
        if(accuracy > current_accuracy):
            print('IMPROVEMENT WAS THERE. SAVE CKPT...')
            current_accuracy = accuracy
print("LEARNING FINISHED! (model: my CNN model)")
print('FINAL ACCURACY: ', current_accuracy)


STEP [ 0 / 53 ] LOSS:  0.6889982223510742
[EPOCH:    1] COST = 0.535172105
EPOCH 1  ACCURACY: 0.6506410256410257
IMPROVEMENT WAS THERE. SAVE CKPT...
STEP [ 0 / 53 ] LOSS:  0.327687531709671
[EPOCH:    2] COST = 0.194589764
EPOCH 2  ACCURACY: 0.8525641025641025
IMPROVEMENT WAS THERE. SAVE CKPT...
STEP [ 0 / 53 ] LOSS:  0.25135231018066406
[EPOCH:    3] COST = 0.15045616
EPOCH 3  ACCURACY: 0.7211538461538461
STEP [ 0 / 53 ] LOSS:  0.23080185055732727


KeyboardInterrupt: 

['NORMAL', 'PNEUMONIA']


데이터 비교 

cuda:0
